Deep Learning Assignment - Conjuctival Melanoma Detection

S. Anusri 
M. Bhagya Sree 
Ankith Mall
G. Srinivas

In [1]:
import os
import shutil
import random
import pandas as pd 
import matplotlib.pyplot as plt
from PIL import Image, ImageOps, ImageEnhance
import numpy as np
import uuid
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [2]:
#Creating the folder for abnormal images
source_dir = "D:\Sem 8\DL\Assignment\image_v2"
normal_dir = os.path.join(source_dir, "normal")
abnormal_dir = os.path.join(source_dir, "abnormal")

os.makedirs(abnormal_dir, exist_ok=True)

abnormal_folders = ["melanoma", "pterygium", "nevus"]
for folder in abnormal_folders:
    images = os.listdir(os.path.join(source_dir, folder))
    for img in images:
        shutil.move(os.path.join(source_dir, folder, img), os.path.join(abnormal_dir, img))

print("Images organized successfully!")


Images organized successfully!


In [3]:
# Function to perform random rotation
def random_rotation(image, max_angle=20):
    angle = random.uniform(-max_angle, max_angle)
    return image.rotate(angle)

# Function to perform random affine transformation
def random_affine(image, translate_range=(0.05, 0.15), scaling_range=(0.9, 0.95)):
    width, height = image.size
    translate_x = random.uniform(-width*translate_range[0], width*translate_range[1])
    translate_y = random.uniform(-height*translate_range[0], height*translate_range[1])
    scaling_factor = random.uniform(scaling_range[0], scaling_range[1])
    return image.transform(image.size, Image.AFFINE, (1, 0, translate_x, 0, 1, translate_y), resample=Image.BICUBIC)

# Function to perform padding
def random_padding(image, padding_range=(0, 10), fill=(0, 0, 0), mode='constant'):
    padding = random.randint(padding_range[0], padding_range[1])
    return ImageOps.expand(image, padding, fill=fill)


# Function to perform color correction
def random_color_correction(image, brightness_range=(0, 0.2), contrast_range=(0, 0.2)):
    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(1 + random.uniform(brightness_range[0], brightness_range[1]))
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(1 + random.uniform(contrast_range[0], contrast_range[1]))
    return image

In [4]:
# Define augmentation functions
augmentation_functions = [
    random_rotation,
    random_affine,
    random_padding,
    random_color_correction
]

In [5]:
def apply_augmentation(input_dir, output_dir, target_count):
    images = os.listdir(input_dir)
    current_count = len(images)
    while current_count < target_count:
        img_name = random.choice(images)
        image_path = os.path.join(input_dir, img_name)
        image = Image.open(image_path).convert("RGB")
        aug_func = random.choice(augmentation_functions)
        augmented_image = aug_func(image)
        new_img_name = f"{current_count}_a.jpg"
        augmented_image.save(os.path.join(output_dir, new_img_name))
        current_count += 1

def shuffle_and_copy_images(input_dir, output_dir, count):
    images = os.listdir(input_dir)
    random.shuffle(images)
    for i in range(count):
        img_name = images[i]
        shutil.copy(os.path.join(input_dir, img_name), os.path.join(output_dir, f"{i}.jpg"))

def split_dataset(input_dir, train_dir, val_dir, test_dir, train_ratio=0.7, val_ratio=0.1):
    images = os.listdir(input_dir)
    random.shuffle(images)
    num_images = len(images)
    num_train = int(train_ratio * num_images)
    num_val = int(val_ratio * num_images)
    train_images = images[:num_train]
    val_images = images[num_train:num_train + num_val]
    test_images = images[num_train + num_val:]
    
    # Copy images to train directory
    for img in train_images:
        shutil.copy(os.path.join(input_dir, img), os.path.join(train_dir, img))
    
    # Copy images to val directory
    for img in val_images:
        shutil.copy(os.path.join(input_dir, img), os.path.join(val_dir, img))
    
    # Copy images to test directory
    for img in test_images:
        shutil.copy(os.path.join(input_dir, img), os.path.join(test_dir, img))

train_dir_normal = os.path.join(source_dir, "train", "normal")
train_dir_abnormal = os.path.join(source_dir, "train", "abnormal")
test_dir_normal = os.path.join(source_dir, "test", "normal")
test_dir_abnormal = os.path.join(source_dir, "test", "abnormal")
val_dir_normal = os.path.join(source_dir, "val", "normal")
val_dir_abnormal = os.path.join(source_dir, "val", "abnormal")

In [6]:
for directory in [train_dir_normal, train_dir_abnormal, test_dir_normal, test_dir_abnormal, val_dir_normal, val_dir_abnormal]:
    os.makedirs(directory, exist_ok=True)

apply_augmentation(normal_dir, normal_dir, 3000)
apply_augmentation(abnormal_dir, abnormal_dir, 3000)
split_dataset(normal_dir, train_dir_normal, val_dir_normal, test_dir_normal, train_ratio=0.7, val_ratio=0.1)
split_dataset(abnormal_dir, train_dir_abnormal, val_dir_abnormal, test_dir_abnormal, train_ratio=0.7, val_ratio=0.1)

shuffle_and_copy_images(train_dir_normal, train_dir_normal, 2100)  # 70% of 3000
shuffle_and_copy_images(train_dir_abnormal, train_dir_abnormal, 2100)  # 70% of 3000
shuffle_and_copy_images(val_dir_normal, val_dir_normal, 300)  # 10% of 3000
shuffle_and_copy_images(val_dir_abnormal, val_dir_abnormal, 300)  # 10% of 3000
shuffle_and_copy_images(test_dir_normal, test_dir_normal, 600)  # 20% of 3000
shuffle_and_copy_images(test_dir_abnormal, test_dir_abnormal, 600)  # 20% of 3000

print("Data augmentation and dataset splitting applied successfully!")

Data augmentation and dataset splitting applied successfully!


In [7]:
val_dir = r"D:\Sem 8\DL\Assignment\image_v2\val"

# Function to copy files from source directory to destination directory
def copy_files(source_dir, dest_dir):
    files = os.listdir(source_dir)
    for file in files:
        source_path = os.path.join(source_dir, file)
        dest_path = os.path.join(dest_dir, file)
        if os.path.isfile(dest_path):
            # If a file with the same name already exists, append a unique identifier to the filename
            file_name, file_extension = os.path.splitext(file)
            unique_id = str(uuid.uuid4())[:8]  # Generate a unique id (8 characters)
            new_file_name = f"{file_name}_{unique_id}{file_extension}"
            dest_path = os.path.join(dest_dir, new_file_name)
        shutil.move(source_path, dest_path)

# Copy contents of 'normal' folder directly into 'val' folder
copy_files(os.path.join(val_dir, "normal"), val_dir)

# Copy contents of 'abnormal' folder directly into 'val' folder
copy_files(os.path.join(val_dir, "abnormal"), val_dir)

# Optionally, you can remove the original 'normal' and 'abnormal' folders
shutil.rmtree(os.path.join(val_dir, "normal"))
shutil.rmtree(os.path.join(val_dir, "abnormal"))

In [8]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [9]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(), #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5, 0.5, 0.5],  #0-1 t0 [-1, 1], formula x - mean/std.dev
                        [0.5, 0.5, 0.5])
])

In [10]:
train_path=r'D:\Sem 8\DL\Assignment\image_v2\train'
test_path=r'D:\Sem 8\DL\Assignment\image_v2\test'


train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path, transform=transformer),
    batch_size=256, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path, transform=transformer),
    batch_size=256, shuffle=True
)

In [11]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [12]:
print(classes)

['abnormal', 'normal']


In [13]:
#CNN Network Class
class ConvNet(nn.Module):
    def __init__(self,num_classes=2):
        super(ConvNet,self).__init__()
        
        #Input shape=(256, 3, 150, 150) 
           #256 - Batch size, 3 RGB channels, 150 -> height, width
        #Output size after applying the convolution filter = ((w-f+2P)/s) + 1), where f = 3 (kernel size or filter size)
           #=> ((150 - 3 + 2)/1) + 1) = (149 + 1 = 150)
        self.conv1=nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)  #Apply the conv2d filter
        #Output shape=(256, 12, 150, 150)
        self.bn1=nn.BatchNorm2d(num_features=12)    #Batch normalization (No.of features = no.of channels), shape still remains same
        #Output shape=(256, 12, 150, 150)
        self.relu1=nn.ReLU() #Add a ReLU or Rectifier Linear Unit function
        #Output shape=(256, 12, 150, 150)
        self.pool=nn.MaxPool2d(kernel_size=2)  #Add Max Pooling layer -> reduce height and width of CNN by a factor of 2
        #Output shape=(256, 12, 75, 75)
        
        #Add one more CNN layer
        self.conv2=nn.Conv2d(in_channels=12, out_channels=20, kernel_size=3, stride=1, padding=1)  #Apply the conv2d filter
        #Output shape=(256, 20, 75, 75)
        self.relu2=nn.ReLU()
        #Output shape=(256, 20, 75, 75)
        
        
        #Add one more CNN layer
        self.conv3=nn.Conv2d(in_channels=20, out_channels=32, kernel_size=3, stride=1, padding=1)  #Apply the conv2d filter
        #Output shape=(256, 32, 75, 75)
        self.bn3=nn.BatchNorm2d(num_features=32)    #Batch normalization (No.of features = no.of channels), shape still remains same
        #Output shape=(256, 32, 75, 75)
        self.relu3=nn.ReLU()
        #Output shape=(256, 32, 75, 75)
        
        #Can add more layers to increase accuracy
        
        #Final fully connected layer:
        self.fc=nn.Linear(in_features=32*75*75, out_features=num_classes) #(32*75*75 -> depth or channels * height * width)
        
        #Feed Forward function
    def forward(self, input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
        #Above output will be in the matrix form with shape(256, 32, 75, 75)
        output=output.view(-1, 32*75*75)
        output=self.fc(output)
        return output

In [14]:
model=ConvNet(num_classes=2).to(device)

In [15]:
#Optimizer and Loss function
optimizer=Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [16]:
num_epochs=3 #Hyperparameter that can be tuned

In [17]:
#Calculate the size of train and test images
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [18]:
print(f"The value of train_count is: {train_count} and the value of test_count is: {test_count}")

The value of train_count is: 8173 and the value of test_count is: 2341


In [19]:
#Model training and save the best model
best_accuracy=0.0
for epoch in range(num_epochs):
    #Evaluation and training on the training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    for i,(images, labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
        optimizer.zero_grad()   #To avoid mixing up of gradients between the batches, zero them out at the start of a new batch
        outputs=model(images)   #Gives the prediction
        loss=loss_function(outputs, labels)  #Computation of loss based on predicted and actual value
        loss.backward() #Back propagation
        optimizer.step() #Updates the weight and bias
        
        train_loss+=loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data, 1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
    train_accuracy/=train_count
    train_loss/=train_count
    
    #Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images, labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
        outputs=model(images)
        _,prediction=torch.max(outputs.data, 1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
        
    test_accuracy/=test_count
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    #Save the best model:
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), 'best_checkpoint.model')
        best_accuracy=test_accuracy

Epoch: 0 Train Loss: tensor(10.3379) Train Accuracy: 0.6527590847913862 Test Accuracy: 0.7817172148654421
Epoch: 1 Train Loss: tensor(0.8172) Train Accuracy: 0.8629634161262695 Test Accuracy: 0.9269542930371636
Epoch: 2 Train Loss: tensor(0.1801) Train Accuracy: 0.9659855622170561 Test Accuracy: 0.9453225117471166


In [20]:
train_path=r'D:\Sem 8\DL\Assignment\image_v2\train'
test_path=r'D:\Sem 8\DL\Assignment\image_v2\test'

In [21]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])
print(classes)

['abnormal', 'normal']


In [22]:
#CNN Network Class
class ConvNet(nn.Module):
    def __init__(self,num_classes=2):
        super(ConvNet,self).__init__()
        
        #Input shape=(256, 3, 150, 150) 
           #256 - Batch size, 3 RGB channels, 150 -> height, width
        #Output size after applying the convolution filter = ((w-f+2P)/s) + 1), where f = 3 (kernel size or filter size)
           #=> ((150 - 3 + 2)/1) + 1) = (149 + 1 = 150)
        self.conv1=nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)  #Apply the conv2d filter
        #Output shape=(256, 12, 150, 150)
        self.bn1=nn.BatchNorm2d(num_features=12)    #Batch normalization (No.of features = no.of channels), shape still remains same
        #Output shape=(256, 12, 150, 150)
        self.relu1=nn.ReLU() #Add a ReLU or Rectifier Linear Unit function
        #Output shape=(256, 12, 150, 150)
        self.pool=nn.MaxPool2d(kernel_size=2)  #Add Max Pooling layer -> reduce height and width of CNN by a factor of 2
        #Output shape=(256, 12, 75, 75)
        
        #Add one more CNN layer
        self.conv2=nn.Conv2d(in_channels=12, out_channels=20, kernel_size=3, stride=1, padding=1)  #Apply the conv2d filter
        #Output shape=(256, 20, 75, 75)
        self.relu2=nn.ReLU()
        #Output shape=(256, 20, 75, 75)
        
        
        #Add one more CNN layer
        self.conv3=nn.Conv2d(in_channels=20, out_channels=32, kernel_size=3, stride=1, padding=1)  #Apply the conv2d filter
        #Output shape=(256, 32, 75, 75)
        self.bn3=nn.BatchNorm2d(num_features=32)    #Batch normalization (No.of features = no.of channels), shape still remains same
        #Output shape=(256, 32, 75, 75)
        self.relu3=nn.ReLU()
        #Output shape=(256, 32, 75, 75)
        
        #Can add more layers to increase accuracy
        
        #Final fully connected layer:
        self.fc=nn.Linear(in_features=32*75*75, out_features=num_classes) #(32*75*75 -> depth or channels * height * width)
        
        #Feed Forward function
    def forward(self, input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
        #Above output will be in the matrix form with shape(256, 32, 75, 75)
        output=output.view(-1, 32*75*75)
        output=self.fc(output)
        return output

In [23]:
checkpoint=torch.load('best_checkpoint.model')
model=ConvNet(num_classes=2)
model.load_state_dict(checkpoint)
model.eval()

ConvNet(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (conv3): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc): Linear(in_features=180000, out_features=2, bias=True)
)

In [24]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(), #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5, 0.5, 0.5],  #0-1 t0 [-1, 1], formula x - mean/std.dev
                        [0.5, 0.5, 0.5])
])

In [25]:
def prediction(img_path, transformer):
    image = Image.open(img_path)
    
    if image.mode != 'RGB':
        image = image.convert('RGB')
    
    image_tensor = transformer(image).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input = Variable(image_tensor)
    model.eval()  # Set the model to evaluation mode
    output = model(input)
    index = output.data.numpy().argmax()
    pred = classes[index]
    return pred

In [26]:
val_path=r'D:\Sem 8\DL\Assignment\image_v2\val'
images_path=glob.glob(val_path+'/*.jpg')

In [27]:
images_path

['D:\\Sem 8\\DL\\Assignment\\image_v2\\val\\0.jpg',
 'D:\\Sem 8\\DL\\Assignment\\image_v2\\val\\0_00f0b3a5.jpg',
 'D:\\Sem 8\\DL\\Assignment\\image_v2\\val\\1.jpg',
 'D:\\Sem 8\\DL\\Assignment\\image_v2\\val\\10.jpg',
 'D:\\Sem 8\\DL\\Assignment\\image_v2\\val\\100.jpg',
 'D:\\Sem 8\\DL\\Assignment\\image_v2\\val\\1009_a.jpg',
 'D:\\Sem 8\\DL\\Assignment\\image_v2\\val\\100_0d62d9f9.jpg',
 'D:\\Sem 8\\DL\\Assignment\\image_v2\\val\\101.jpg',
 'D:\\Sem 8\\DL\\Assignment\\image_v2\\val\\1014_a.jpg',
 'D:\\Sem 8\\DL\\Assignment\\image_v2\\val\\1018_a.jpg',
 'D:\\Sem 8\\DL\\Assignment\\image_v2\\val\\101_21913017.jpg',
 'D:\\Sem 8\\DL\\Assignment\\image_v2\\val\\102.jpg',
 'D:\\Sem 8\\DL\\Assignment\\image_v2\\val\\1021_a.jpg',
 'D:\\Sem 8\\DL\\Assignment\\image_v2\\val\\1023_a.jpg',
 'D:\\Sem 8\\DL\\Assignment\\image_v2\\val\\1023_a_e704bd19.jpg',
 'D:\\Sem 8\\DL\\Assignment\\image_v2\\val\\1024_a.jpg',
 'D:\\Sem 8\\DL\\Assignment\\image_v2\\val\\1028_a.jpg',
 'D:\\Sem 8\\DL\\Assignment\\

In [28]:
val_dict={}
for i in images_path:
    val_dict[i[i.rfind('/')+1:]]=prediction(i, transformer)

In [29]:
val_dict

{'D:\\Sem 8\\DL\\Assignment\\image_v2\\val\\0.jpg': 'normal',
 'D:\\Sem 8\\DL\\Assignment\\image_v2\\val\\0_00f0b3a5.jpg': 'abnormal',
 'D:\\Sem 8\\DL\\Assignment\\image_v2\\val\\1.jpg': 'normal',
 'D:\\Sem 8\\DL\\Assignment\\image_v2\\val\\10.jpg': 'normal',
 'D:\\Sem 8\\DL\\Assignment\\image_v2\\val\\100.jpg': 'normal',
 'D:\\Sem 8\\DL\\Assignment\\image_v2\\val\\1009_a.jpg': 'abnormal',
 'D:\\Sem 8\\DL\\Assignment\\image_v2\\val\\100_0d62d9f9.jpg': 'abnormal',
 'D:\\Sem 8\\DL\\Assignment\\image_v2\\val\\101.jpg': 'normal',
 'D:\\Sem 8\\DL\\Assignment\\image_v2\\val\\1014_a.jpg': 'normal',
 'D:\\Sem 8\\DL\\Assignment\\image_v2\\val\\1018_a.jpg': 'normal',
 'D:\\Sem 8\\DL\\Assignment\\image_v2\\val\\101_21913017.jpg': 'abnormal',
 'D:\\Sem 8\\DL\\Assignment\\image_v2\\val\\102.jpg': 'normal',
 'D:\\Sem 8\\DL\\Assignment\\image_v2\\val\\1021_a.jpg': 'abnormal',
 'D:\\Sem 8\\DL\\Assignment\\image_v2\\val\\1023_a.jpg': 'normal',
 'D:\\Sem 8\\DL\\Assignment\\image_v2\\val\\1023_a_e704bd19